In [1]:
from datasets import load_dataset
from transformers import MT5Tokenizer, MT5ForConditionalGeneration, MT5Config
import torch
from noise_functions.MT5NoiseFunction import MT5NoiseFunction
from noise_functions.MT6NoiseFunction import MT6NoiseFunction

In [1]:
from MT6 import MT6
from transformers import MT5TokenizerFast, T5ForConditionalGeneration

tok = MT5TokenizerFast.from_pretrained("nikodallanoce/mt5-cc4-vanilla-32k-5")
model = MT6(
        MT5Config(num_layers=6, d_kv=64, d_model=512, num_heads=8, d_ff=2048, vocab_size=len(tok), max_length=128,
                  tie_word_embeddings=True))
model.train(False)
# model = MT6(MT5Config(vocab_size=len(tok), tie_word_embeddings=True))
model.num_parameters()

NameError: name 'MT5Config' is not defined

In [2]:
from MT6 import MT6
from transformers import MT5TokenizerFast, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM

#tok = MT5TokenizerFast.from_pretrained("nikodallanoce/mt5-cc4-vanilla-32k-5")
tok = AutoTokenizer.from_pretrained("google/t5-v1_1-small")
cuda_dev = "cpu"

model = AutoModelForSeq2SeqLM.from_pretrained(
        "/data/n.dallanoce/weights/mt5_ft_en-fr_M1F1_t5_mbart/checkpoint-85000",
        )
model = model.to(cuda_dev)
model.train(False)
model.eval()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(32100, 512, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(32100, 512, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 512)
      (layers): ModuleList(
        (0-5): 6 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,)

In [20]:
dataset = load_dataset("cc100", lang="en",
                       cache_dir="/data/n.dallanoce/cc100/huggingface",
                       split=f"train[{4096}:{4096 * 2}]",
                       verification_mode='no_checks')

In [58]:
index = 167
sent = dataset[index]['text']
src_sent, tgt_sent = MT5NoiseFunction(0.35, 3).compute(sent, seed=index)
print(f"original: {sent} \n \nsource: {src_sent}")

original: wants, and he will be so lost. His whole life is in chairs. (Having a
 
 
source: wants, and <extra_id_0> His whole life <extra_id_1> chairs. (Having a


In [62]:
from noise_functions.MT5NoiseFunction import MT5NoiseFunction

index = 2
#sent = dataset[index]['text']
sent = "Officials say an escape route is complete and workers trapped for over two weeks will be freed soon."

src_sent, tgt_sent = MT5NoiseFunction(0.35, 3).compute(sent, seed=index)
#src_sent = sent + "</s> "+ src_sent
print(f"original: {sent} \n \nsource: {src_sent}")

original: Officials say an escape route is complete and workers trapped for over two weeks will be freed soon. 
 
source: Officials say an <extra_id_0> route <extra_id_1> workers trapped <extra_id_2> two weeks will be freed soon.


In [66]:
tok_sent = tok(src_sent, return_tensors="pt")
tok_sent["input_ids"] = tok_sent["input_ids"].to(cuda_dev)
tok_sent["attention_mask"] = tok_sent["attention_mask"].to(cuda_dev)
sequence_ids = model.generate(**tok_sent, max_new_tokens=128, num_beams=4)
sequences = tok.batch_decode(sequence_ids, skip_special_tokens=False)
print(f" prediction: {sequences} \n \n source: {src_sent} \n \n label: {tgt_sent}")

/data/n.dallanoce/anaconda3/envs/deeptorch/lib/python3.9/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


 prediction: ['<pad><extra_id_0> emergency <extra_id_1> for <extra_id_2> within <extra_id_3></s>'] 
 
 source: Officials say an <extra_id_0> route <extra_id_1> workers trapped <extra_id_2> two weeks will be freed soon. 
 
 label: <extra_id_0> escape <extra_id_1> is complete and <extra_id_2> for over <extra_id_3>


In [9]:
to_translate = "translate English to French: Two days ago I lost my cat."
tok_sent = tok(to_translate, return_tensors="pt", max_length=128, padding="max_length", truncation=True)
tok_sent["input_ids"] = tok_sent["input_ids"].to(cuda_dev)
#tok_sent["attention_mask"] = tok_sent["attention_mask"].to(cuda_dev)
sequence_ids = model.generate(**tok_sent, max_new_tokens=128, num_beams=5,
                              #bos_token_id=tok.bos_token_id, eos_token_id=tok.eos_token_id
                              )
sequences = tok.batch_decode(sequence_ids, skip_special_tokens=True)
print(sequences)

['Il y a deux jours, j’ai perdu mon chat.']


In [10]:
tok_sent["input_ids"]

tensor([[13959,  1566,    12,  2379,    10,  2759,   477,   977,    27,  1513,
            82,  1712,     5,     1,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  